# **Working with time series**
---
 
 
- Copyright (c) Lukas Gonon, 2024. All rights reserved

- Author: Lukas Gonon <l.gonon@imperial.ac.uk>

- Platform: Tested on Windows 10 with Python 3.9

**Time series:**

- any set of data indexed by time (stock price, number of sales, running activity/fitness,....)
- frequency varies (month by month for macroeconomics down to milliseconds for high frequency)

Importing the library `pandas` for data analysis

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
## We consider the US air passenger data, publicly available at https://www.kaggle.com/chirag19/air-passengers

df = pd.read_csv("data/AirPassengers.csv")
df.head()

In [ ]:
df['Month'] = pd.to_datetime(df['Month'], format='%Y-%m')
df.index = df['Month']

In [ ]:
df.index.names = ['Date']
del df['Month']
df.head()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df)
plt.title("Number of Passengers")
plt.show()

## Stationarity?

Stationarity of a time series refers to the potential fact that some patters repeat themselves over time.

*Can you think of some examples of time repetitive patters?*

In [ ]:
rolling_period = 7 ## (in days)

rolling_mean = df.rolling(rolling_period).mean()
rolling_std = df.rolling(rolling_period).std()

plt.plot(df, color="blue",label="Original Passenger Data")

plt.plot(rolling_mean, color="red", label="Rolling Mean Passenger Number")

plt.plot(rolling_std, color="black", label = "Rolling Standard Deviation in Passenger Number")

plt.title("Passenger Time Series, Rolling Mean, Standard Deviation")

plt.legend(loc="best")
plt.show()

**Question:** does the data look stationary?

The Dickey-Fuller test is a statistical test taking non-stationarity as null hypothesis

In [ ]:
#pip install statsmodels

In [ ]:
from statsmodels.tsa.stattools import adfuller

test_df = adfuller(df, autolag="AIC")


test_df_output = pd.DataFrame({"Values":[test_df[0],test_df[1],test_df[2],test_df[3], test_df[4]['1%'], test_df[4]['5%'], test_df[4]['10%']], 
                               "Metric":["Test Statistics","p-value","No. of lags used","Number of observations used", 
                                         "critical value (1%)", "critical value (5%)", "critical value (10%)"]})


print(test_df_output)

## Autocorrelation

Given a time series $(X_t)$, the autocovariance function, describing the linear dependence between data points, is given by
$$
\gamma(s,t) := \mathrm{cov}(X_s,X_t) = \mathbb{E}\Big[(X_s - \mu_s)(X_t-\mu_t)\Big],
$$
where $\mu_t := \mathbb{E}[X_t]$,
and the autocorrelation function is defined as
$$
\rho(s,t) := \frac{\gamma(s,t)}{\sqrt{\gamma(s,s)\gamma(t,t)}} \in [-1,1].
$$

*Note*: 
If the series is weakly stationary in the sense that:
- $\mathbb{E}[X_t]$ is constant,
- $\mathbb{V}[X_t]$ is finite,
- $\gamma(s,t) = \gamma(s+h,t+h)$,
then
$\gamma(t, t+h) = \gamma(0,h) =:\overline{\gamma}(h)$ and 
$$
\rho(t,t+h) = \frac{\widetilde{\gamma}(h)}{\sqrt{\gamma(0,0)\gamma(0,0)}} = \frac{\widetilde{\gamma}(h)}{\widetilde{\gamma(0)}} =: \widetilde{\rho}(h).
$$


In [ ]:
lags = [1, 3, 5, 7, 9]

for l in lags:
    print("Lag (in months): ", l, "|| value: ", df['#Passengers'].autocorr(lag=l))
    
    
longlags = np.arange(1, 50)
autocorrels = [df['#Passengers'].autocorr(lag=l) for l in longlags]

plt.plot(longlags, autocorrels, 'b')
plt.title("Autocorrelation")
plt.show()

## Decomposition

**Question:** Does the data evolve additively or multiplicatively over time?

- Additive evolution: 
$$
X_{t+1} = \textrm{Base} + \textrm{Trend}_{t} + \textrm{Seasonality}_{t} + \varepsilon_{t}
$$

- Multiplicative evolution
$$
X_{t+1} = \textrm{Base} * \textrm{Trend}_{t} * \textrm{Seasonality}_{t} * \varepsilon_{t}
$$

Here $(\varepsilon_t)_{t}$ is a *white noise*, namely a sequence of uncorrelated random variable with mean zero and finite variance (standard example is Gaussian white noise).

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

### Additive decomposition

In [ ]:
seasonal_decompose(df['#Passengers'], model='additive', period=7).plot()
plt.title("Additive decomposition")
plt.show()

### Multiplicative decomposition

In [ ]:
seasonal_decompose(df['#Passengers'], model='multiplicative', period=7).plot()
plt.title("Multiplicative decomposition")
plt.show()

**Question:** What do you think?

## Forecasting

**Question:** Given an observed time series (in the past), can we predict (at least part of) the future?

*Method:* We set a date to split the data into a training set and a test set

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
## We select (1-prop)% of the data as training and prop% as test.

prop = .05
datesplit = df.index[-int(prop*len(df.index))]
datesplit

### Set the training data

In [ ]:
trainset = df[df.index < datesplit].copy()
trainset.rename(columns={'#Passengers': 'traindata'}, inplace=True)
trainset.tail()

### Set the test data

In [ ]:
testset = df[df.index >= datesplit].copy()
testset.rename(columns={'#Passengers': 'testdata'}, inplace=True)
testset.head()

### Plot the two sets (training + test)

In [ ]:
plt.plot(trainset, color = "black", label="training set")
plt.plot(testset, color = "blue", label="test set")
plt.title("Train/Test split")
plt.ylabel("Number of passengers")
plt.legend(loc="best")
plt.xlabel('Date')
plt.show()

There exist many (!!!) mathematical models to describe time series.
We consider here a standard one, called **ARIMA** (Auto Regressive Integrated Moving Average).
Here and below, $(\varepsilon_{t})$ always denotes a (Gaussian) white noise.

- **AR(p) model:**
$$
X_{t}  = \sum_{i=1}^{p}\phi_{i} X_{t-i} + \varepsilon_{t},
$$
or equivalently 
$$
\Phi(B) X_{t} = \varepsilon_{t},
$$
where $\Phi(z) = \sum_{i=1}^{p}\phi_{i} z^{i}$ and the backshift operator $B$ acts as $B X_{t} := X_{t-1}$.

- **MA(q) model:**
$$
X_{t}  = \varepsilon_{t} + \sum_{j=1}^{q}\psi_{i} \varepsilon_{t-j}
 = \Psi(B)\varepsilon_{t},
$$
where $\Psi(z) := 1 + \sum_{j=1}^{q}\psi_{j} z^{j}$.
*Note that an MA model is always stationary.*


- **ARMA(p,q) model:**
$$
X_{t}  = \sum_{i=1}^{p}\phi_{i} X_{t-i} + \varepsilon_{t} + \sum_{j=1}^{q}\psi_{i} \varepsilon_{t-j}\\
$$
equivalently,
$$
\Phi(B) X_{t} = \Psi(B)\varepsilon_{t}.
$$



- **ARIMA(p,q,d) model:**
$$
\Phi(B) (1-B)^{d}X_{t} = \Psi(B)\varepsilon_{t}.
$$

The reason is as follows:
$\nabla X_t := X_{t} - X_{t-1} = (1-B)X_t$ and
$\nabla^2 X_t := \nabla(X_{t} - X_{t-1}) = (X_{t} - X_{t-1}) - (X_{t-1} - X_{t-2})
 = X_{t} - 2X_{t-1}+ X_{t-2} = (1-B)^2X_t$,
and so on.

In [ ]:
#pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima
model = auto_arima(trainset, trace=True, error_action='ignore', suppress_warnings=True)
model.fit(trainset)
forecast = model.predict(n_periods=len(testset))
forecast = pd.DataFrame(forecast,index = testset.index,columns=['Prediction'])

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
rms = sqrt(mean_squared_error(testset,forecast))
print("RMSE: ", rms)